In [ ]:
import import_ipynb
from phase1_stategraph_to_code import node_to_code_app

d:\agentagent\AgentAgent\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dict_for_node = {
    "planner": {
        "schema_info": """OverallState:
      type: TypedDict
      fields:
      - name: input
        type: str
      - name: plan
        type: List[str]
      - name: past_steps
        type: List[Tuple]
      - name: response
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "OverallState",
    "output_schema":"OverallState",
    "description":"Plan step generates a plan based on the input using llm structured output functionality, stores it to the plan field",
    "function_name": "plan_step"
    },
    "math_node":{
        "schema_info": """MathState:
      type: TypedDict
      fields:
      - name: a
        type: int
      - name: b
        type: int
      - name: response
        type: int""",
    "input_schema": "MathState",
    "output_schema":"MathState",
    "description":"Gets the two numbers a and b from the state, and returns the sum of a and b., stores it to the response field",
    "function_name": "math_step"
    },
    "interrupt_node":{
        "schema_info": """RequirementAnalysisState:
      type: TypedDict
      fields:
      - name: user_input
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "RequirementAnalysisState",
    "output_schema":"RequirementAnalysisState",
    "description":"Analyze the user input, see if they require any human intervention, if yes, return the human intervention required, else return no human intervention required",
    "function_name": "interrupt_step"
    },
    "decision_node":{
        "schema_info": """RequirementAnalysisState:
      type: TypedDict
      fields:
      - name: user_input
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "RequirementAnalysisState",
    "output_schema":"RequirementAnalysisState",
    "description":"Analyze the user input which contain the requirements for building a product, acts as a decision node if more info is needed or not",
    "function_name": "decision_step"
    },
    "code_node":{
        "schema_info": """CodeWriterState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "CodeWriterState",
    "output_schema":"RequiremenCodeWriterStatetAnalysisState",
    "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code",
    "function_name": "code_step"
    },
    "weather_node":{
        "schema_info": """WeatherNodeState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "WeatherNodeState",
    "output_schema":"WeatherNodeState",
    "description":"This node analyzes the user_query, makes tool calls to retrieve relevant information and answer the query",
    "function_name": "weather_step",
    },
    "stock_node":{
        "schema_info": """StockState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "StockState",
    "output_schema":"StockState",
    "description":"This node analyzes the user_query related to finance/stock-markets, makes tool calls to retrieve relevant information and answer the query",
    "function_name": "stock_step",
    }
}

for output in node_to_code_app.stream(dict_for_node["stock_node"], stream_mode="updates"
    ):
        print(output)

{'identify_node': {'node_type': 'ai', 'messages': [HumanMessage(content='\nYou are provided with the following information about the node:\n<SchemaInfo>\nStockState:\n      type: TypedDict\n      fields:\n      - name: user_query\n        type: str\n      - name: execution_result\n        type: str\n</SchemaInfo>\n<InputSchema>\nStockState\n</InputSchema>\n<OutputSchema>\nStockState\n</OutputSchema>\n<Description>\nThis node analyzes the user_query related to finance/stock-markets, makes tool calls to retrieve relevant information and answer the query\n</Description>\n<FunctionName>\nstock_step\n</FunctionName>\n\nBelow is the skeleton of the function that you need to implement:\ndef stock_step(state:StockState) -> StockState:\n    """This node analyzes the user_query related to finance/stock-markets, makes tool calls to retrieve relevant information and answer the query"""\n    # Implement the function to meet the description.\n\nthe state is of type StockState and the function is of 